# Forms

Up to now, we have only implemented views that retrieve data from the database. Suppose that we want to allow users to update profiles (later we will talk about restricting this capability to certain users)

Django has the `forms.ModelForm` class for 

`clubmanager/forms.py`
```python
from django import forms
from clubmanager.models import Member

class MemberForm(forms.ModelForm):
    model = Member
```


## The view

`views.py`
```python
from clubmanager.forms import MemberForm

def member_update(request,member_id):
    form = MemberForm()
    # now we need to work out whether data is being submitted or not
```